In [1]:
import datajoint as dj
import numpy as np
import time

In [121]:
ta3p100 = dj.create_virtual_module("ta3p100","microns_ta3p100")
schema = dj.schema("microns_ta3p100")


In [125]:
len(ta3p100.PairSkeletonDistFiltered()),len(ta3p100.FinalContactTest())




(3144023, 0)

In [ ]:
#practice doing correlations

#Matrix we will have in the end: n_post_syn by n_post_syn 
#how do we generate each point in the matrix
#    - correlations of 2 n_presyn vectors


In [ ]:
a = np.array([[1,3,5,10],[2,4,6,8],[1,3,5,10],[2,4,6,8]])
b = np.array([[5,6,7,8],[9,10,11,12],[5,6,7,8],[9,10,11,12]])
print(np.corrcoef(a,b))

In [ ]:

a = np.array([[1,3,5,10],[2,4,6,8],[1,3,5,10],[2,4,6,8]])
b = np.array([[5,6,7,8],[9,10,11,12],[5,6,7,8],[9,10,11,12]])


from scipy.stats import pearsonr
x = [1, 2, 3, 4, 5]
corr = [2, 4, 6, 8, 10]
corr_2 = [3 , 5 ,7, 2, 3]
corr_value, p_value = pearsonr(x, corr_2)
print(corr_value)

In [ ]:
#Another way using numpy

print(np.corrcoef(x, corr_2)[0, 1])
np.corrcoef(x, corr_2)

In [ ]:
#CONCLUSION: this is the final way to do the correlations with multiple neurons
np.corrcoef(a)

In [ ]:
ta3p100.NeuriteTargetsRevised()

In [41]:
presyn = ta3p100.NeuriteTargetsRevised().fetch("segment_id")

#presyn.indexof[]

search_list = [648518346341351518,648518346341351514]


#testing the sorting
%timeit np.searchsorted(presyn, search_list)
#print([i for i, e in enumerate(haystack) if e in st])
#%timeit np.where(presyn in item)

3.3 µs ± 134 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [63]:
presyn

array([648518346341351467, 648518346341351503, 648518346341351508, ...,
       648518346349507986, 648518346349507993, 648518346349507995])

In [60]:
#####THIS METHOD WON'T TELL YOU IF SOMETHING IS IN THE LIST OR NOT, 
###JUST TELLS YOU WHAT INDEX IT WILL INSERT it in (so if not in there will just pick closest index)

search_list = [648518346341351518,648518346341351514,presyn[101],10,presyn[0]]

start_time = time.time()
x = np.searchsorted(presyn, search_list)  
print(x)
print(f"Total time: {time.time()-start_time}")
"""gpomg tp use this method because it is only one got working"""
start_time = time.time()
y = [i for i,z in enumerate(presyn) if z in search_list]
print(y)
print(f"Total time: {time.time()-start_time}")
start_time = time.time()
presyn_set = set(presyn)
y = [i for i,z in enumerate(presyn_set) if z in search_list]
print(y)
print(f"Total time: {time.time()-start_time}")

presyn_set = set(presyn)
y = np.where(search_list > 3)
print(y)
print(f"Total time: {time.time()-start_time}")


#x_other = np.where((presyn in [search_list[0]]).any())
#x_other



[  6   5 101   0   0]
Total time: 0.0010292530059814453
[0, 5, 6, 101]
Total time: 0.0390629768371582
[26, 47, 50, 679]
Total time: 0.04352235794067383


TypeError: '>' not supported between instances of 'list' and 'int'

In [71]:
(presyn.any())# in search_list)

True

In [66]:
x = np.arange(9.).reshape(3, 3)
presyn[np.where(presyn in search_list)]


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [57]:
presyn_set = set(presyn)
start_time = time.time()
presyn[1000] in presyn_set
print(f"Total time: {time.time()-start_time}")
start_time = time.time()
presyn[1000] in presyn
print(f"Total time: {time.time()-start_time}")

Total time: 0.0001590251922607422
Total time: 0.0002727508544921875


In [ ]:
"""Pseudo code for correlations 

1) Starting from the Pre-Post table, proj(
presyn,
postsyn,
total_contact_conversion,
total_contact_density,
mean_volume

2) Option A: just do only for common presyn pairs
a. Pull down the whole pre-post data table and put into pandas table
b. Generate (on datajoint) all of the unique pairs

c. For each of the unique pairs: 
    1) Get the conversion rate and presyns for segment A and B
    2) Do an intersection of presyns
        - if no intersectionn then save group
    3) Get the conversion rates that match the intersected presyns
    4) Calculate pearson coefficient
    5) Save in dictionary

d. Write dictionary to database
- Write no overlap to database



"""

In [ ]:
np.searchsorted(presyn, 648518346341351518)

In [ ]:
ta3p100.ContactTestSharedContact()

In [3]:
targets = (dj.U("postsyn") & ta3p100.ContactTestSharedContact).proj(segment_id="postsyn") - ta3p100.SegmentExclude
info = targets * targets.proj(segment_b='segment_id') & 'segment_id < segment_b'
info

segment_pairs = info.fetch()
segment_pairs

array([(648518346341352006, 648518346341353019),
       (648518346341352223, 648518346341353019),
       (648518346341353019, 648518346341353186),
       (648518346341352006, 648518346341353186),
       (648518346341352223, 648518346341353186),
       (648518346341353019, 648518346341353574),
       (648518346341353186, 648518346341353574),
       (648518346341352006, 648518346341353574),
       (648518346341352223, 648518346341353574),
       (648518346341352006, 648518346341352223)],
      dtype=[('segment_id', '<i8'), ('segment_b', '<i8')])

In [ ]:
ta3p100.ContactPrePostTest()

In [81]:
df

,segmentation,presyn,postsyn,total_contact_conversion,total_contact_density
0,2,648518346341351467,648518346341352006,6.85714,0.154386
1,2,648518346341351467,648518346341352223,7.50000,0.178105
2,2,648518346341351467,648518346341353019,2.66667,0.044457
3,2,648518346341351467,648518346341353186,2.33333,0.042276
4,2,648518346341351467,648518346341353574,5.50000,0.120932
5,2,648518346341351503,648518346341352006,5.60000,0.169934
6,2,648518346341351503,648518346341352223,3.75000,0.133215
7,2,648518346341351503,648518346341353019,2.00000,0.061719
8,2,648518346341351503,648518346341353186,3.50000,0.109838
9,2,648518346341351503,648518346341353574,4.60000,0.128707


In [120]:
#download the projection of the Neurites Table
import pandas as pd

prepost_data = ta3p100.ContactPrePostTestReal.proj("presyn",
                    "postsyn","total_contact_conversion","total_contact_density").fetch()

df = pd.DataFrame(prepost_data)
df
#how to restrict a table
# df['presyn'] == 648518346341351467
#how to do two value restrictions: df[df['first_name'].notnull() & (df['nationality'] == "USA")]

start_time = time.time()
postsyn1 = 648518346341353019
postsyn2 = 648518346341352223

df_1 = df[df["postsyn"]==postsyn1][:5]
df_2 = df[df["postsyn"]==postsyn2][1:]
#how you restrict a table by the values in the other table
df_1_common = df_1[df_1["presyn"].isin(df_2["presyn"])].sort_values(by=['presyn'])
df_2_common = df_2[df_2["presyn"].isin(df_1["presyn"])].sort_values(by=['presyn'])
#how to extract the data
#postsyns = restr_df["presyn"].to_numpy()
df_1_common_conversion = df_1[df_1["presyn"].isin(df_2["presyn"])].sort_values(by=['presyn'])["total_contact_conversion"].to_numpy()
df_2_common_conversion = df_2[df_2["presyn"].isin(df_1["presyn"])].sort_values(by=['presyn'])["total_contact_conversion"].to_numpy()

#postsyns = restr_df["presyn"].to_numpy()
df_1_common_density = df_1[df_1["presyn"].isin(df_2["presyn"])].sort_values(by=['presyn'])["total_contact_density"].to_numpy()
df_2_common_density = df_2[df_2["presyn"].isin(df_1["presyn"])].sort_values(by=['presyn'])["total_contact_density"].to_numpy()
    
#calculate the coefficient
from scipy.stats import pearsonr

corr_value, p_value = pearsonr(df_1_common_conversion, df_2_common_conversion)
corr_value, p_value = pearsonr(df_1_common_density, df_2_common_density)
print(corr_value)
print(f"Total time for correlation = {time.time()-start_time}")

-0.6026168953755794
Total time for correlation = 0.016229867935180664
